In [18]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 크롬 드라이버 설정
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

# 직무별 URL과 직무 이름 매핑
categories = [
    {
        'role': '머신러닝 엔지니어',
        'url': 'https://www.wanted.co.kr/wdlist/518/1634?country=kr&job_sort=job.recommend_order&years=-1&selected=1634&locations=all'
    },
    {
        'role': '데이터 엔지니어',
        'url': 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&selected=655&locations=all'
    },
    {
        'role': '데이터 사이언티스트',
        'url': 'https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=job.recommend_order&years=-1&selected=1024&locations=all'
    },
    {
        'role': '빅데이터 엔지니어',
        'url': 'https://www.wanted.co.kr/wdlist/518/1025?country=kr&job_sort=job.recommend_order&years=-1&selected=1025&locations=all'
    }
]

# 모든 직무 데이터를 저장할 리스트
all_job_data = []

for category in categories:
    print(f"Fetching data for role: {category['role']}")
    driver.get(category['url'])
    time.sleep(5)  # 페이지 로드 대기

    # 스크롤 끝까지 내리기
    scroll_pause_time = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 공고 리스트 가져오기
    try:
        job_elements = wait.until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__next"]/div[3]/div[2]/ul/li'))
        )
    except Exception as e:
        print(f"Error locating job elements for {category['role']}: {e}")
        job_elements = []

    # 공고 데이터 추출
    for job_element in job_elements:
        try:
            job_title = job_element.find_element(By.XPATH, './div/a/div[2]/span[1]').text
            company_name = job_element.find_element(By.XPATH, './div/a/div[2]/span[2]').text
            location = job_element.find_element(By.XPATH, './div/a/div[2]/span[3]').text
            job_url = job_element.find_element(By.XPATH, './div/a').get_attribute('href')

            # 데이터 저장 (직무 추가)
            all_job_data.append({
                'role': category['role'],
                'title': job_title,
                'company': company_name,
                'location': location,
                'url': job_url
            })
        except Exception as e:
            print(f"Error extracting job data for {category['role']}: {e}")

# CSV 파일로 저장
csv_file = 'wanted_jobs.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['role', 'title', 'company', 'location', 'url'])
    writer.writeheader()
    writer.writerows(all_job_data)

print(f"Data saved to {csv_file}. Total records: {len(all_job_data)}")

# # 드라이버 종료
# driver.quit()


Fetching data for role: 머신러닝 엔지니어
Fetching data for role: 데이터 엔지니어
Fetching data for role: 데이터 사이언티스트
Fetching data for role: 빅데이터 엔지니어
Data saved to wanted_jobs.csv. Total records: 810


In [19]:
import pandas as pd

# CSV 파일 경로 (다운로드한 파일의 경로를 지정)
file_path = 'wanted_jobs.csv'

# CSV 파일 읽기
jobs_data = pd.read_csv(file_path)

# 데이터 확인
print(jobs_data.head())  # 첫 몇 개의 행 출력
print(f"총 데이터 수: {len(jobs_data)}")  # 데이터 개수 확인

        role                                     title  company  \
0  머신러닝 엔지니어  Sr. Machine Learning Engineer (HPCNT AI)   하이퍼커넥트   
1  머신러닝 엔지니어              AI Research Engineer (2년 이상)  폴스타헬스케어   
2  머신러닝 엔지니어                              멀티모달 AI 엔지니어       로민   
3  머신러닝 엔지니어                                   AI 엔지니어     카이헬스   
4  머신러닝 엔지니어                            AI Lead (5년이상)      비글즈   

            location                                 url  
0  서울 강남구 · 경력 3-13년  https://www.wanted.co.kr/wd/254260  
1  서울 서초구 · 경력 2년 이상  https://www.wanted.co.kr/wd/254283  
2  서울 서초구 · 경력 2-10년  https://www.wanted.co.kr/wd/254250  
3  서울 강남구 · 경력 3년 이상  https://www.wanted.co.kr/wd/254173  
4  경기 성남시 · 경력 5-15년  https://www.wanted.co.kr/wd/253998  
총 데이터 수: 810


In [20]:
import pandas as pd

# 중복값 식별
duplicates = jobs_data[jobs_data.duplicated(subset=['title', 'company'], keep=False)]

# 중복값 출력
print("Duplicates:")
print(duplicates)

# 중복값 제거
jobs_data_deduplicated = jobs_data.drop_duplicates(subset=['title', 'company'], keep='first')

# 중복 제거 후 데이터 저장
deduplicated_file_path = 'wanted_jobs_deduplicated.csv'
jobs_data_deduplicated.to_csv(deduplicated_file_path, index=False, encoding='utf-8-sig')

print(f"Deduplicated data saved to: {deduplicated_file_path}")


Duplicates:
          role                                   title  \
2    머신러닝 엔지니어                            멀티모달 AI 엔지니어   
10   머신러닝 엔지니어                                  AI 개발자   
21   머신러닝 엔지니어                        머신러닝엔지니어 (3년 이상)   
22   머신러닝 엔지니어               NLP Researcher / Engineer   
31   머신러닝 엔지니어                      Senior ML Engineer   
..         ...                                     ...   
805  빅데이터 엔지니어                       Data Engineer(Jr)   
806  빅데이터 엔지니어                          Data Scientist   
807  빅데이터 엔지니어                                데이터 엔지니어   
808  빅데이터 엔지니어                           Data Engineer   
809  빅데이터 엔지니어  [인텔리전스랩스] 리서치 사이언티스트 (Computer Vision)   

                       company            location  \
2                           로민   서울 서초구 · 경력 2-10년   
10                       미니레코드    서울 송파구 · 경력 2-6년   
21                        엠트리센   서울 금천구 · 경력 3년 이상   
22   올거나이즈코리아(Allganize Korea)   서울 강남구 · 경력 1-10년   
31                   

In [21]:
# Get the total number of rows in the deduplicated data
total_rows_deduplicated = jobs_data_deduplicated.shape[0]

total_rows_deduplicated

580

In [22]:
import pandas as pd

# CSV 파일 읽기
file_path = 'wanted_jobs_deduplicated.csv'  # 파일 경로를 실제 경로로 변경하세요
jobs_data = pd.read_csv(file_path)

# 역할(직무)과 키워드 매핑 (기존 매핑 확장 가능)
role_keywords = {
    "데이터 분석가": [
        ['데이터', '분석'],    
        ['data', 'analyst'],  
        ['db', '분석'],      
        ['데이터', '애널리스트'],
        ['buisness', 'analyst'],
        ['analyst'],   
        ['분석'],     
        ['데이터 분석'], 
        ['data analyst'],
        ['DB'],
        ['데이터']
    ],
    "데이터 엔지니어": [
        ['데이터', '엔지니어'],    
        ['data', 'engineer'],  
        ['db', '엔지니어'],      
        ['db', 'engineer'],
        ['dw'],
        ['etl'],
        ['spark']
    ],
    "AI 서비스 개발자": [
        ['a', 'i', '개발'],
        ['인공지능', '개발'],
        ['a', 'i', '엔지니어'],
        ['a', 'i', 'engineer'],
        ['a', 'i', '연구'],
        ['a', 'i', 'architect'],
        ['딥', '러닝'],
        ['deep', 'learning'],
        ['machine', 'learning'],
        ['a', 'i', 'researcher'],
        ['a', 'i', 'solution'], 
        ['a', 'i', '전문가'],
        ['생성'],
        ['imaging'],
        ['vision'],
        ['nlp']
    ],
    "챗봇 개발자": [
        ['챗봇', '개발'],
        ['a', 'i', '개발'],
        ['llm', '개발'],
        ['react', '개발'], 
        ['프론트', '개발'],
        ['bot']
    ],
    "클라우드 엔지니어": [
        ['클라우드', '엔지니어'],
        ['cloud', 'engineer'],
        ['cloud', '엔지니어'],
        ['클라우드', 'engineer'],
        ['cloud', 'architect'],
        ['클라우드', '아키텍트'],
        ['클라우드', '운영'],
        ['시스템', '엔지니어'],
        ['클라우드', '개발'],
        ['cloud', '개발'],
        ['서버', '엔지니어'],
        ['네트워크', '엔지니어'],
        ['network', '엔지니어'],
        ['네트워크', 'engineer'],
        ['인프라', '엔지니어'],
        ['cloud', 'sa'],
        ['aws'],
        ['azure'],
        ['gcp']
    ],
    "API 개발자": [
        ['API'], 
        ['rest'], 
        ['soap'], 
        ['graphql'], 
        ['api 개발'], 
        ['api 설계'], 
        ['endpoint'],
        ['백엔드', '개발'],
        ['Back-end'],
        ['server']
    ],
    "머신러닝 엔지니어": [
        ['머신러닝'], 
        ['Machine Learning'], 
        ['ML'], 
        ['tensorflow'], 
        ['scikit-learn'], 
        ['keras'], 
        ['모델링'], 
        ['머신러닝 개발'],
        ['머신', '러닝', '엔지니어'],
        ['machine', 'learning', 'engineer'],
        ['ml', 'engineer'],
        ['머신', '러닝', 'engineer'],
        ['ml', '엔지니어'],
        ['machine', 'learning', '엔지니어'],
        ['머신', '러닝', '개발'],
        ['ml', '개발'],
        ['machine', 'learning', '개발'],
        ['a', 'i', '개발'],
        ['모델'],
        ['predictive']
    ],
    "데이터 사이언티스트": [
        ['사이언티스트'], 
        ['Scientist'], 
        ['Data Scientist'], 
        ['분석 모델'], 
        ['머신러닝'], 
        ['예측 분석'], 
        ['모델 평가'], 
        ['Data Science'],
        ['statistics'],
        ['R'],
        ['python', 'analysis']
    ]
}

# 역할(직무)을 제목(title) 기준으로 분류하는 함수
def assign_role(title):
    """
    제목에 키워드가 포함되어 있는지 확인하고, 해당 역할을 반환.
    키워드 그룹의 모든 키워드가 포함된 경우 해당 역할을 반환.
    키워드가 포함되지 않은 경우 '기타' 반환.
    """
    title = title.lower()  # 제목을 소문자로 변환
    for role, keyword_groups in role_keywords.items():
        for keywords in keyword_groups:
            if all(keyword.lower() in title for keyword in keywords):  # 키워드를 소문자로 변환하여 비교
                return role
    return "기타"  # 어떤 역할에도 해당되지 않는 경우

# 역할 분류 적용
jobs_data['role'] = jobs_data['title'].apply(assign_role)

# role이 "기타"인 데이터만 재분류
misc_data = jobs_data[jobs_data['role'] == '기타']
if not misc_data.empty:
    jobs_data.loc[misc_data.index, 'role'] = misc_data['title'].apply(assign_role)

# 결과를 새로운 CSV 파일로 저장
output_file_path = 'wanted_jobs_categorized_roles_updated.csv'  # 결과를 저장할 파일 경로
jobs_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# 데이터 요약 정보 출력
role_counts = jobs_data['role'].value_counts()
print(f"총 데이터 개수: {len(jobs_data)}")
print(f"분류된 데이터가 저장되었습니다: {output_file_path}")
print("\n업데이트된 역할별 데이터 개수:")
print(role_counts)


총 데이터 개수: 580
분류된 데이터가 저장되었습니다: wanted_jobs_categorized_roles_updated.csv

업데이트된 역할별 데이터 개수:
role
AI 서비스 개발자    229
데이터 분석가       132
기타             62
데이터 사이언티스트     47
머신러닝 엔지니어      46
데이터 엔지니어       40
API 개발자        17
클라우드 엔지니어       5
챗봇 개발자          2
Name: count, dtype: int64


In [23]:
# 기존 데이터프레임에서 location 컬럼을 지역과 경력으로 나누기
# location 컬럼이 "경기 성남시 · 경력 7-15년" 형식으로 되어 있다고 가정

def split_location(location):
    """
    location 컬럼을 '지역'과 '경력'으로 나누는 함수.
    """
    if pd.isna(location):  # location 값이 NaN일 경우 처리
        return pd.Series([None, None])  # NaN일 경우 둘 다 None 반환
    parts = location.split(' · ')
    if len(parts) == 2:
        return pd.Series([parts[0].strip(), parts[1].strip()])  # ['지역', '경력'] 반환
    elif len(parts) == 1:  # 경력이 없는 경우
        return pd.Series([parts[0].strip(), None])  # 지역만 반환
    return pd.Series([None, None])  # 예외 처리: None 반환

# location 컬럼 분리 적용
jobs_data[['지역', '경력']] = jobs_data['location'].apply(split_location)

# 결과를 새로운 CSV 파일로 저장
output_file_path = 'wanted_jobs_categorized_with_location.csv'  # 결과 저장 경로
jobs_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# 데이터 요약 정보 출력
print(f"총 데이터 개수: {len(jobs_data)}")
print(f"지역 및 경력 컬럼이 추가된 데이터가 저장되었습니다: {output_file_path}")


총 데이터 개수: 580
지역 및 경력 컬럼이 추가된 데이터가 저장되었습니다: wanted_jobs_categorized_with_location.csv
